# 데이콘 소득 예측 경진대회

- 예측이지만, binary classification 문제

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import pandas as pd
import sys
from catboost import CatBoostClassifier
from tqdm.auto import tqdm

In [58]:
train=pd.read_csv("./data/train_cat_modified.csv")
train.head()

,age,workclass,fnlwgt,education_num,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,target,education1,education2,education3,country_group1,country_group2,country_group3,country_group4,age_group
0,3.465736,Private,309513,12,Married-civ-spouse,Craft-repair,White,Male,0,0,40,0,0,1,0,0,0,1,0,1
1,3.496508,Private,205469,10,Married-civ-spouse,Exec-managerial,White,Male,0,0,40,1,0,1,0,0,0,1,0,1
2,3.828641,Private,149949,10,Married-civ-spouse,Craft-repair,White,Male,0,0,40,0,0,1,0,0,0,1,0,1
3,3.135494,Private,193090,13,Never-married,Adm-clerical,White,Female,0,0,30,0,0,1,0,0,0,1,0,0
4,4.007333,Private,60193,9,Divorced,Adm-clerical,White,Female,0,0,40,0,0,1,0,0,0,1,0,1


In [59]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             16707 non-null  float64
 1   workclass       16707 non-null  object 
 2   fnlwgt          16707 non-null  int64  
 3   education_num   16707 non-null  int64  
 4   marital_status  16707 non-null  object 
 5   occupation      16707 non-null  object 
 6   race            16707 non-null  object 
 7   sex             16707 non-null  object 
 8   capital_gain    16707 non-null  int64  
 9   capital_loss    16707 non-null  int64  
 10  hours_per_week  16707 non-null  int64  
 11  target          16707 non-null  int64  
 12  education1      16707 non-null  int64  
 13  education2      16707 non-null  int64  
 14  education3      16707 non-null  int64  
 15  country_group1  16707 non-null  int64  
 16  country_group2  16707 non-null  int64  
 17  country_group3  16707 non-null 

In [60]:
train[['workclass','marital_status','occupation','race','sex']]=train[['workclass','marital_status','occupation','race','sex']].astype('category')

cat_features = ['workclass','marital_status','occupation','race','sex']


In [61]:
test=pd.read_csv("./data/test_cat_modified.csv")
test.head()

,age,workclass,fnlwgt,education_num,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,education1,education2,education3,country_group1,country_group2,country_group3,country_group4,age_group
0,3.850148,Private,304133,10,Married-civ-spouse,Exec-managerial,White,Male,0,0,45,0,1,0,0,0,1,0,1
1,3.526361,Self-emp-inc,154227,10,Never-married,Sales,White,Male,0,0,75,0,1,0,0,0,1,0,1
2,3.433987,Local-gov,158291,13,Never-married,Craft-repair,White,Male,8614,0,40,0,1,0,0,0,1,0,1
3,3.332205,Private,183155,13,Married-civ-spouse,Prof-specialty,White,Male,0,0,55,0,1,0,0,0,1,0,1
4,3.988984,Local-gov,182543,10,Widowed,Adm-clerical,White,Female,0,0,40,0,1,0,0,0,0,1,1


In [62]:
test[['workclass','marital_status','occupation','race','sex']]=test[['workclass','marital_status','occupation','race','sex']].astype('category')

In [63]:
train_x=train.drop('target',axis=1)
train_y=train['target']

In [ ]:
# GridSearchCV 를 이용해 모델들을 최적화시켜줍니다.
from tqdm.auto import tqdm


classifier = CatBoostClassifier(verbose=True,task_type="GPU",cat_features=cat_features)

params = {
        'n_estimators':[50,100,200,500],
        'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [5, 7, 9],
        #'one_hot_max_size'=[3,12,16]
        
}


grid_result = classifier.grid_search(params,
                      X=train_x,
                      y=train_y,
                      plot=True
                      )


In [13]:
grid_result

{'params': {'depth': 10,
  'l2_leaf_reg': 7,
  'iterations': 50,
  'learning_rate': 0.1},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,

In [17]:
classifier = CatBoostClassifier(depth=10,l2_leaf_reg=7,learning_rate=0.1,iterations=50,verbose=True,task_type="GPU")

In [27]:
classifier.fit(train_x, train_y)
y_pred = classifier.predict(test)

0:	learn: 0.5782835	total: 26ms	remaining: 1.27s
1:	learn: 0.4974512	total: 51ms	remaining: 1.22s
2:	learn: 0.4403311	total: 77.8ms	remaining: 1.22s
3:	learn: 0.4010782	total: 103ms	remaining: 1.19s
4:	learn: 0.3734772	total: 129ms	remaining: 1.16s
5:	learn: 0.3524944	total: 153ms	remaining: 1.12s
6:	learn: 0.3366008	total: 175ms	remaining: 1.07s
7:	learn: 0.3234800	total: 195ms	remaining: 1.02s
8:	learn: 0.3139181	total: 218ms	remaining: 991ms
9:	learn: 0.3068403	total: 240ms	remaining: 959ms
10:	learn: 0.3004368	total: 262ms	remaining: 928ms
11:	learn: 0.2948660	total: 281ms	remaining: 890ms
12:	learn: 0.2906459	total: 301ms	remaining: 858ms
13:	learn: 0.2870160	total: 324ms	remaining: 832ms
14:	learn: 0.2837001	total: 344ms	remaining: 802ms
15:	learn: 0.2813862	total: 365ms	remaining: 775ms
16:	learn: 0.2783866	total: 388ms	remaining: 753ms
17:	learn: 0.2761841	total: 409ms	remaining: 726ms
18:	learn: 0.2741549	total: 429ms	remaining: 700ms
19:	learn: 0.2716957	total: 450ms	remainin

In [28]:
test

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,education1,education2,education3,country_group1,...,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving,Male,no-working,baeksu
0,3.850148,304133,10,0,0,45,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,3.526361,154227,10,0,0,75,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,3.433987,158291,13,8614,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3.332205,183155,13,0,0,55,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,3.988984,182543,10,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15076,3.401197,182926,10,15024,0,40,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
15077,3.663562,176101,13,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
15078,3.871201,242406,7,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
15079,3.784190,100584,9,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [29]:
y_pred

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [30]:
y_pred.sum()

2960

In [31]:
submission = pd.read_csv('./data/sample_submission.csv')
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [32]:
submission['target']=y_pred
submission

,id,target
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
15076,15076,1
15077,15077,1
15078,15078,0
15079,15079,0


In [33]:
submission.to_csv("./catboost_dummy.csv",index=False)